# Demo Run

In [1]:
import os
import sys
import time
import warnings
import anndata as ad
import scanpy as sc
from pathlib import Path

sys.path.append(str(Path(os.getcwd()).resolve().parents[0]))

In [2]:
import tardis

In [3]:
adata_file_path = (
    "/Users/kemalinecik/git_nosync/tardis/data/"
    "dataset_subset_age_1.h5ad"
)
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
metadata_of_interest = "age"

In [4]:
adata = ad.read_h5ad(adata_file_path)
adata.obs["age"] = adata.obs["age"].astype(str).astype("category")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tardis.MyModel.setup_anndata(adata, layer=None, batch_key="concatenated_integration_covariates")
    vae = tardis.MyModel(
        adata, 
        n_hidden = 512,
        n_layers=3, 
        n_latent=10, 
        gene_likelihood="nb",
        dropout_rate = 0.1
    )
    vae.train(max_epochs=200)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 3/200:   1%|          | 2/200 [00:13<21:54,  6.64s/it, v_num=1, train_loss_step=176, train_loss_epoch=228]

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()
sc.pp.neighbors(adata, n_neighbors = 30, use_rep="X_scVI")
sc.tl.umap(adata, min_dist=0.2)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sc.pl.umap(
        adata, 
        color=[metadata_of_interest, "cell_type", "concatenated_integration_covariates"], 
        ncols=3,
        frameon=False,
        title="",
        legend_fontsize="xx-small"
    )